## Necessary Modules

In [3]:
import os
from glob import glob
import sys
sys.path.append('..')

from datetime import datetime, timedelta
from itertools import product

import pandas as pd
import numpy as np

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String, Time, DateTime, Numeric, BigInteger
from sqlalchemy import select, insert, update, delete

import yfinance as yf
from litedb import Litedata
Yahoodata = Litedata('yahoo')

## Testing on sectypes

### Reading symbollist CSVs

In [4]:
symbolcsvlist = glob(os.path.join(Yahoodata.data_path, 'lite_*.csv'))

for csvfile in symbolcsvlist:
    dflist_sectype = pd.read_csv(csvfile, index_col='symbol', header=0)
    dflist_sectype = dflist_sectype.sort_values(['sector', 'symbol'])
    dflist_sectype.to_csv(csvfile)

### Remove disconnected & obsolete tables

In [ ]:
#tablelist = Yahoodata.engine.table_names()
#droplist = [name for name in tablelist if name[:6] == 'midcap']
#print(droplist)

#with Yahoodata.engine.connect() as con:
#    for name in droplist:
#        stmt = f'DROP TABLE {name}'
#        con.execute(stmt)

### Remove symbol

In [5]:
#removelist = ['1589.HK', 'CERN', 'PBCT']

#for symbol in removelist:
#    Yahoodata.remove_symbol(symbol)

### Inserting all symbols from a security type

In [ ]:
#sectypelist_new = ['hkstock']
#metadict = {sectype: Yahoodata.get_symboldict_csv(sectype) for sectype in sectypelist_new}
#for sectype in sectypelist_new:
#    metadict[sectype] = Yahoodata.get_symboldict_csv(sectype)
#    Yahoodata.insert_from_symboldict(metadict[sectype])

In [ ]:
#with Yahoodata.engine.connect() as con:
#    meta = MetaData()
#    insertlist = [{'vendor': 'yahoo', 'date': datetime(2020, 3, 23, 16, 0, 0),
#                    'op': 21659.55, 'hi': 22062.19, 'lo': 21659.55, 'cl': 21696.13, 'vol': 3478982700,
#                    'adj_op': 21659.55, 'adj_hi': 22062.19, 'adj_lo': 21659.55, 'adj_cl': 21696.13, 'adj_vol': 3478982700,
#                    'div': 0, 'split': 0, 'update': datetime.now()}] 
#    table_ohlc = Table('Indices_^HSI', meta, autoload=True, autoload_with=Yahoodata.engine)
#    insertquery = insert(table_ohlc)
#    con.execute(insertquery, insertlist)

### Parsing HK new symbols

In [ ]:
newfile = os.path.join(Yahoodata.data_path, 'hsci-change.xlsx')
newxl = pd.ExcelFile(newfile)

dfchg = pd.DataFrame()
newdate = datetime(2020, 7, 27)

for name in newxl.sheet_names[:-1]:
    dfsector = newxl.parse(name)
    dfsector['Date'] = pd.to_datetime(dfsector['Date'])
    dfsecnew = dfsector[dfsector['Date'] >= newdate]
    dfsecnew.rename(columns={'Symbol': 'symbol'}, inplace=True)
    dfchg = pd.concat([dfchg, dfsecnew], axis=0)
    
dfchg.sort_values(['Date', 'act', 'sector', 'symbol'], ascending=(False, True, True, True), inplace=True)
print(dfchg.info())

dfadd = dfchg[dfchg['act'] == 'add']
dfdel = dfchg[dfchg['act'] == 'delete']
print(dfadd.info())
print(dfadd['symbol'].nunique())
print(dfdel.info())
print(dfdel['symbol'].nunique())

dfadd.set_index('symbol', inplace=True)
dfadd.sort_index(inplace=True)
dfdel.set_index('symbol', inplace=True)
dfdel.sort_index(inplace=True)

In [ ]:
dfnew = dfadd.copy()
dfnew['ex_code'] = 'HKSE'
dfnew['sectype'] = 'hkstock'
dfnew['currency'] = 'HKD'
dfnew['industry'] = 'none'
dfnew['startdate'] = '2022-06-10'
dfnew['enddate'] = '2022-06-17'
dfnew.drop(['Date', 'act'], axis=1, inplace=True)
dfnew = dfnew[['ex_code', 'sectype', 'currency', 'name', 'sector', 'industry', 'startdate', 'enddate']]

In [ ]:
#hkfile = os.path.join(Yahoodata.data_path, 'lite_hkstock.csv')
#dfhk = pd.read_csv(hkfile, header=0, index_col='symbol')

#for code in dfnew.index:
#    if code in dfhk.index:
#        dfnew.drop(code, axis=0, inplace=True)
        
#dfhk = pd.concat([dfhk, dfnew], axis=0)
#dfhk.to_csv(hkfile)